In [1]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from openai import OpenAI
from langchain.tools import tool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.messages import HumanMessage, SystemMessage
import tt

# 1

In [2]:
class CreateMarketingContent:
    """Creates content for LinkedIn Marketing"""

    def __init__(self, subject):
        self.llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")
        self.subject = subject
        self.llm_chains = []

    def get_information_about_company(self):
        information = f"""
* Learn2Earn AB is a leading training company specializing in IT, cloud solutions, architecture, ML and AI training
* They provide students with cutting-edge knowledge and skills essential for success in the tech world
* They have a new course that has just been released
* The subject of the new course is: {self.subject}
* You are the company social media manager, responsible for advertising the course on LinkedIn
* Your task is to create a single image ad
* It's important that the ad conveys that the company is extremely professional and trustworthy
* You will create a title, a subtitle, and a description for the ad
"""
        return information

    @staticmethod
    def create_template(text):
        template = text
        return template

    @staticmethod
    def create_prompt(template):
        prompt = ChatPromptTemplate.from_template(template)
        return prompt

    def create_llm_chain(self, prompt, output_key):
        chain = LLMChain(llm=self.llm, prompt=prompt, output_key=output_key)
        self.llm_chains.append(chain)

    def create_sequential_chain(self):
        sequential_chain = SequentialChain(
            chains=self.llm_chains,
            input_variables=["information"],
            output_variables=["title", "subtitle", "description", "additional_recommendations"]
        )

        information = self.get_information_about_company()
        result = sequential_chain(information)
        return result

    def create_background_image(self):
        client = OpenAI()
        prompt = f"Create an image that is perfect as background for an ad about a course on {self.subject}"
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="hd",
            style="natural"
        )

        return response

    @staticmethod
    def print_results(result):
        print(f"Title:\n{result['title']}\n")
        print(f"Subtitle:\n{result['subtitle']}\n")
        print(f"Description:\n{result['description']}\n")
        print(f"Additional Recommendations:\n{result['additional_recommendations']}\n")

    def run(self):
        template1 = self.create_template(tt.tt1)
        prompt1 = self.create_prompt(template1)
        self.create_llm_chain(prompt=prompt1, output_key="title")

        template2 = self.create_template(tt.tt2)
        prompt2 = self.create_prompt(template2)
        self.create_llm_chain(prompt=prompt2, output_key="subtitle")

        template3 = self.create_template(tt.tt3)
        prompt3 = self.create_prompt(template3)
        self.create_llm_chain(prompt=prompt3, output_key="description")

        template4 = self.create_template(tt.tt4)
        prompt4 = ChatPromptTemplate.from_template(template4)
        self.create_llm_chain(prompt=prompt4, output_key="additional_recommendations")

        result = self.create_sequential_chain()

        return result


# 2

In [3]:
@tool
def web_searcher(query: str) -> str:
    """Takes a query, performs a web search and returns the search result"""

    search = SerpAPIWrapper(search_engine="google")
    search_result = search.run(query)

    return search_result

In [4]:
def generate_title(subject: str, topics: str) -> str:
    """The function generates a title for a course"""

    system_message = ("system", f"You are a very powerful assistant")
    user_message = ("user",
                    "Create a title for a course about {subject} that covers {topics}. Use the web_searcher_tool for inspiration")

    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=1)
    tools = [web_searcher]
    prompt = ChatPromptTemplate.from_messages(
        [system_message, user_message, MessagesPlaceholder(variable_name="agent_scratchpad")])

    agent = create_openai_functions_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools)

    query_result = agent_executor.invoke({"subject": subject, "topics": topics})

    return query_result["output"]


In [5]:
# subject = "Python basics"
# topics = "intro, variables, print statement, conditional statements, iterations, functions, oop"
# r = generate_title(subject, topics)
# print(r)

# 3

In [6]:
def generate_outline_course(subject: str, topics: str) -> str:
    """The function generates an outline for a course"""

    system_message = ("system", f"You are a very skilled curriculum designer")
    user_message = ("user",
                    "Create an outline for a course with the title {subject} that covers {topics}. The course is for professional programmers but with no experience in {subject}. Use the web_searcher tool for inspiration")

    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=1)
    tools = [web_searcher]
    prompt = ChatPromptTemplate.from_messages(
        [system_message, user_message, MessagesPlaceholder(variable_name="agent_scratchpad")])

    agent = create_openai_functions_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools)

    result = agent_executor.invoke({"subject": subject, "topics": topics})

    return result["output"]

In [7]:
# subject = "Python basics"
# topics = "intro, variables, print statement, conditional statements, iterations, functions, oop"
# r = generate_outline_course(subject, topics)
# print(r)

# 4

In [8]:
def generate_outline_course_non_tech(subject: str, topics: str) -> str:
    """The function generates an outline for a course"""

    system_message = ("system", f"You are a very skilled curriculum designer")
    user_message = ("user",
                    "Create an outline for a course with the title {subject} that covers {topics}. The course is for non tech people. Use the web_searcher tool for inspiration")

    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=1)
    tools = [web_searcher]
    prompt = ChatPromptTemplate.from_messages(
        [system_message, user_message, MessagesPlaceholder(variable_name="agent_scratchpad")])

    agent = create_openai_functions_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools)

    result = agent_executor.invoke({"subject": subject, "topics": topics})

    return result["output"]

In [9]:
# subject = "Python basics"
# topics = "intro, variables, print statement, conditional statements, iterations, functions, oop"
# r = generate_outline_course_non_tech(subject, topics)
# print(r)

# 5

In [10]:
def create_exercises(subject: str):
    model = ChatOpenAI(model="gpt-3.5-turbo")
    system_message = "You are a skilled Python developer and you have been teaching Python for many years"
    human_message = f"Give me 5 {subject} exercises"
    messages = [
        SystemMessage(content=system_message),
        HumanMessage(content=human_message)
    ]

    response = model.invoke(messages, temperature=0)

    return response.content

In [11]:
# r = create_exercises("dictionaries")
# print(r)